# Indice

* Mi viaje en programacion: procedural -> orientada a objetos -> funcional
* MapReduce como simplificacion de la computacion distribuida
* Reglas para distribuir el trabajo:
    * Funciones puras (sin mutabilidad)
    * Sin Excepciones (Monads: Option y Try)
* Practica local
    * map
    * flatMap
    * reduce
    * Mutabilidad y condiciones de carrera
    * manejo de excepciones
* En streaming
    * akka-streams
    * dataflow / scio
    * apache spark

# Mi viaje: Basic -> C# -> Scala

## Procedural
GOTO?

## OOP: clases y objetos
perfecto, hasta que tenemos que usar varios procesadores. Lock/mutex nos restringen el uso de multiples procesadores

## Funcional
Incomodo y dificil de razonar en un principio. El mejor camino para sacarle el mejor performance a multiples procesadores

# MapReduce como simplificacion de la computacion distribuida
¿Como procesarias bigdata, si solo pudieras usar personas para procesar todo?

* Enseñar a cada uno como procesar los datos (funcion/algoritmia)
* Fraccionar el trabajo (en chunks) para poder distribuir parte de los datos a cada usuario (shuffle)
* Enviar datos a los usuarios para que los procesen y nos devuelva el resultado (Map o Reduce)
* Juntar todos los datos

## Map: N -> N (chunk -> chunk)
Entrar N registros y sale la misma cantidad

## Reduce: N -> M (chunk -> dato)
Entra N registros y sale M registros. Muchas veces sale 1 solo registro por chunk


# Reglas para distribuir el trabajo

## Funciones puras
Cada dato debe ser calculado independiente de los datos anteriores, para evitar dependencias y condiciones de carrera

## No usar excepciones
Se deben capturar los errores sin que esos arrojen una excepcion al sistema (o uno de los trabajadores podria caerse y dejar el trabajo en un estado incierto).
Los monad (Option/Try) nos facilitan esta tarea (su explicacion en unos minutos)

# Practica local

In [1]:
// Carguemos dependencias
import $ivy.`org.scala-lang.modules:scala-parallel-collections_2.13:1.0.3`
import scala.collection.parallel.CollectionConverters._
import scala.util.{Try, Success, Failure}

import $ivy.$                                                             

import scala.collection.parallel.CollectionConverters._

import scala.util.{Try, Success, Failure}

## map

In [2]:
val duplica: Int => Int = { i => i * 2 }
def triplica(v: Int): Int = v * 3

(1 to 10).map(duplica)
(1 to 10).map(triplica)

duplica: Int => Int = ammonite.$sess.cmd1$Helper$$Lambda$2385/1240126282@3075689a
defined function triplica
res1_2: IndexedSeq[Int] = Vector(2, 4, 6, 8, 10, 12, 14, 16, 18, 20)
res1_3: IndexedSeq[Int] = Vector(3, 6, 9, 12, 15, 18, 21, 24, 27, 30)

In [3]:
def aumenta(n:Int)(v: Int): Int = n * v

(1 to 10).map(aumenta(2))
(1 to 10).map(aumenta(3))

defined function aumenta
res2_1: IndexedSeq[Int] = Vector(2, 4, 6, 8, 10, 12, 14, 16, 18, 20)
res2_2: IndexedSeq[Int] = Vector(3, 6, 9, 12, 15, 18, 21, 24, 27, 30)

In [4]:
val cuatriplica = aumenta(4)(_)

(1 to 10).map(cuatriplica)

cuatriplica: Int => Int = ammonite.$sess.cmd3$Helper$$Lambda$2459/1076319808@c89cf46
res3_1: IndexedSeq[Int] = Vector(4, 8, 12, 16, 20, 24, 28, 32, 36, 40)

## flatMap

In [5]:
val censo = List (
    "onka,panda,quimera", // casa 1
    "monty", // casa 2
    "canaima", // casa 3
    "obi,max" // casa 4
)

censo.map { perros => perros.split(",") }

censo: List[String] = List("onka,panda,quimera", "monty", "canaima", "obi,max")
res4_1: List[Array[String]] = List(
  Array("onka", "panda", "quimera"),
  Array("monty"),
  Array("canaima"),
  Array("obi", "max")
)

In [6]:
censo.map { perros => perros.split(",") }.flatten

res5: List[String] = List(
  "onka",
  "panda",
  "quimera",
  "monty",
  "canaima",
  "obi",
  "max"
)

In [7]:
censo.flatMap { perros => perros.split(",") }

res6: List[String] = List(
  "onka",
  "panda",
  "quimera",
  "monty",
  "canaima",
  "obi",
  "max"
)

## reduce

In [8]:
val edades = {    
    val rnd = new java.util.Random
    
    (1 to 10).map { i =>
        val value = rnd.nextInt % 75
        
        if (value > 0)
            value
        else
            value * -1
    }
}

val maximo: (Int, Int) => Int = { case (a, b) => if(a > b) a else b }
val minimo: (Int, Int) => Int = { case (a, b) => if(a < b) a else b }
val suma: (Int, Int) => Int = _ + _

edades.reduce(maximo)
edades.reduce(minimo)
edades.reduce(suma)

edades: IndexedSeq[Int] = Vector(48, 29, 71, 7, 2, 12, 29, 31, 28, 16)
maximo: (Int, Int) => Int = ammonite.$sess.cmd7$Helper$$Lambda$2607/1053818651@69e51f04
minimo: (Int, Int) => Int = ammonite.$sess.cmd7$Helper$$Lambda$2608/120085331@388e9f34
suma: (Int, Int) => Int = ammonite.$sess.cmd7$Helper$$Lambda$2609/1320308301@49015052
res7_4: Int = 71
res7_5: Int = 2
res7_6: Int = 273

In [9]:
edades.par.reduce(maximo)
edades.par.reduce(minimo)
edades.par.reduce(suma)

res8_0: Int = 71
res8_1: Int = 2
res8_2: Int = 273

## Mutabilidad y condiciones de carrera

In [10]:
val numTuplas = 8
val testTupla =
    (1 to numTuplas).map { _ => (1, 2) } ++
    (1 to numTuplas).map { _ => (2, 3) }

numTuplas: Int = 8
testTupla: IndexedSeq[(Int, Int)] = Vector(
  (1, 2),
  (1, 2),
  (1, 2),
  (1, 2),
  (1, 2),
  (1, 2),
  (1, 2),
  (1, 2),
  (2, 3),
  (2, 3),
  (2, 3),
  (2, 3),
  (2, 3),
  (2, 3),
  (2, 3),
  (2, 3)
)

In [11]:
// funcion pura
def suma(a: Int, b: Int) = a + b

testTupla.map { case (a,b) => suma(a, b) }

defined function suma
res10_1: IndexedSeq[Int] = Vector(
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5
)

In [12]:
testTupla.par.map { case (a,b) => suma(a, b) }

res11: collection.parallel.immutable.ParSeq[Int] = ParVector(3, 3, 3, 3, 3, 3, 3, 3, 5, 5, 5, 5, 5, 5, 5, 5)

In [13]:
// funcion impura
var acumulado = 0
def acumula(a: Int) = {
    acumulado = acumulado + a
    
    acumulado
}

testTupla.map { case (a,b) => acumula(a) }

acumulado: Int = 15
defined function acumula
res12_2: IndexedSeq[Int] = Vector(
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  10,
  12,
  14,
  16,
  18,
  20,
  22,
  24
)

In [14]:
// reseteamos acumulador
acumulado = 0
// resultado no es igual al anterior
testTupla.par.map { case (a,b) => acumula(a) }

res13_1: collection.parallel.immutable.ParSeq[Int] = ParVector(1, 2, 4, 5, 5, 6, 6, 7, 2, 3, 5, 7, 9, 11, 13, 15)

## Manejo de excepciones

In [15]:
val intText = List("1", "2", "3", "cuatro", "5", "-1")

intText: List[String] = List("1", "2", "3", "cuatro", "5", "-1")

In [16]:
intText.map { t => t.toInt }

: 

In [17]:
intText.map{ i => try {
        i.toInt
    } catch {
        // que devuelvo si no puedo procesar
        case ex: Throwable => -1
    }
}

res16: List[Int] = List(1, 2, 3, -1, 5, -1)

In [18]:
intText.map { i => Try { i.toInt } }

res17: List[Try[Int]] = List(
  Success(1),
  Success(2),
  Success(3),
  Failure(java.lang.NumberFormatException: For input string: "cuatro"),
  Success(5),
  Success(-1)
)

In [19]:
intText.map { i => Try { i.toInt }.toOption }
intText.flatMap { i => Try { i.toInt }.toOption }

res18_0: List[Option[Int]] = List(
  Some(1),
  Some(2),
  Some(3),
  None,
  Some(5),
  Some(-1)
)
res18_1: List[Int] = List(1, 2, 3, 5, -1)

# En streaming

## akka-streams
vamos al notebook akka-streams!

## [apache beam](https://beam.apache.org) / [scio](https://github.com/spotify/scio) y [apache spark](https://spark.apache.org)
ambos siguen los mismos principios de programacion funcional.
las [UDF](https://spark.apache.org/docs/latest/api/scala/org/apache/spark/sql/expressions/UserDefinedFunction.html) de scala usan una funcion como entrada 

# ¡Muchas gracias a todos!